In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import time
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from keras.models import Sequential, Model
from keras.layers import Conv2D, Conv2DTranspose, Reshape, Input, Dropout, UpSampling2D
from keras.layers import Flatten, BatchNormalization, Dense, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
import random

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
## General constants 
# Shape of the dataset and generator output images.
image_shape = (64, 64, 3)

# Dataset batch size
batch_size = 100

# Number of channels of the dataset and generator output images.
# n_channels = 3

# How many epochs should train last.
epochs = 16000

## Dataset related constants 
#  Dataset path
dataset_path = "./logos_dataset/class/"

latent_dimensions = 100

display_interval = 20

# # Last epoch during previous training (in case training
# # has continued from the checkpoint).
# last_epoch = 0

In [ ]:
# Defining load_dataset function
def load_dataset(dataset_path, batch_size, image_shape):
    dataset_generator = ImageDataGenerator()
    dataset_generator = dataset_generator.flow_from_directory(
        dataset_path, target_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size,
        class_mode=None)
    
    return dataset_generator

In [ ]:
# Building the generator model. 
def generator_model():
    generator = Sequential()

    generator.add(Dense(units=4 * 4 * 512,
                        kernel_initializer='glorot_uniform',
                        input_shape=(1, 1, 100)))
    generator.add(Reshape(target_shape=(4, 4, 512)))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=256, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=128, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=64, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=3, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(Activation('tanh'))

    optimizer = Adam(lr=0.00015, beta_1=0.5)
    generator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=None)

    print('generator')
    generator.summary()
    
    return generator

In [ ]:
# Building the discriminator model. 
def discriminator_model(image_shape):
    discriminator = Sequential()
    discriminator.add(Conv2D(filters=64, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform',
                             input_shape=(image_shape)))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(filters=128, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    discriminator.add(BatchNormalization(momentum=0.5))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(filters=256, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    discriminator.add(BatchNormalization(momentum=0.5))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(filters=512, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    discriminator.add(BatchNormalization(momentum=0.5))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Flatten())
    discriminator.add(Dense(1))
    discriminator.add(Activation('sigmoid'))

    optimizer = Adam(lr=0.0002, beta_1=0.5)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=None)

    print('discriminator')
    discriminator.summary()
    
    return discriminator

In [ ]:
# Displaying and saving generated images as '.png' 
def save_generated_images(generated_images, epoch, batch_number):
    plt.figure(figsize=(8, 8), num=2)
    gs1 = gridspec.GridSpec(8, 8)
    gs1.update(wspace=0, hspace=0)

    for i in range(64):
        ax1 = plt.subplot(gs1[i])
        ax1.set_aspect('equal')
        image = generated_images[i, :, :, :]
        image += 1
        image *= 127.5
        fig = plt.imshow(image.astype(np.uint8))
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

    plt.tight_layout()
    save_name = 'generated images/generatedSamples_epoch' + str(
        epoch + 1) + '_batch' + str(batch_number + 1) + '.png'

    plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
    plt.pause(0.0000000001)
    plt.show()

In [ ]:
# Building adversarial model(gan)
def train_dcgan(batch_size, epochs, image_shape, dataset_path):
    generator = construct_generator()
    discriminator = construct_discriminator(image_shape)

    gan = Sequential()
    
    discriminator.trainable = False
    gan.add(generator)
    gan.add(discriminator)

    if os.path.exists("./output/generator_weights.h5"):
        print('loaded generator model weights')
        generator.load_weights('./output/generator_weights.h5')

    if os.path.exists("./output/discriminator_weights.h5"):
        print('loaded discriminator model weights')
        discriminator.load_weights('./output/discriminator_weights.h5')

    optimizer = Adam(lr=0.001, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer,
                metrics=None)

    # Creating a dataset Generator 
    dataset_generator = load_dataset(dataset_path, batch_size, image_shape)

    # Total number of images used is 1073
    number_of_batches = int(1073 / batch_size)

    # Variables that will be used to plot the losses from the discriminator and
    # the adversarial models
    adversarial_loss = np.empty(shape=1)
    discriminator_loss = np.empty(shape=1)
    batches = np.empty(shape=1)

    # plot updates inside for loop
    plt.ion()

    current_batch = 0

In [ ]:
# setting current_batch to 1
current_batch = 1

In [ ]:
# defining generator loss function
def gen_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# defining dicriminator loss function
def disc_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
 # Creating dataset generator 
dataset_generator = load_dataset(dataset_path, batch_size, image_shape)

In [ ]:
# Total number of images used is 1073
number_of_batches = int(1073 / batch_size)

# Variables that will be used to plot the losses from the discriminator and
# the adversarial models
adversarial_loss = np.empty(shape=1)
discriminator_loss = np.empty(shape=1)
batches = np.empty(shape=1)

In [ ]:
# creating generator
generator = generator_model()

# creating discriminator
discriminator = discriminator_model(image_shape)

In [ ]:
# creating gan model
gan = Sequential()

discriminator.trainable = False
gan.add(generator)
gan.add(discriminator)

optimizer = Adam(lr=0.00015, beta_1=0.5)
gan.compile(loss='binary_crossentropy', optimizer=optimizer,
            metrics=None)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
#Defining a utility function to display the generated images

def display_images():
    r,c = 4,4
    noise = np.random.normal(0, 1, size=(current_batch_size,) + (1, 1, 100))
    generated_images = generator.predict(noise)
    
    #Scaling the generated images
    generated_images = 0.5 * generated_images + 0.5
    
    fig, axs = plt.subplots(r, c)
    count = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(generated_images[count, :,:,])
            axs[i,j].axis('off')
            count +=1
    plt.show()
    plt.close()

In [ ]:
# Training DcGAN for n epochs
for epoch in range(epochs):
    print("Epoch " + str(epoch+1) + "/" + str(epochs) + " :")
    
    for batch_number in range(number_of_batches):
        start_time = time.time()
        # Get the current batch and normalize the images between -1 and 1
        real_images = dataset_generator.next()
        real_images /= 127.5
        real_images -= 1

        # The last batch is smaller than the other ones, so we need to
        # take that into account
        current_batch_size = real_images.shape[0]

        # Generate noise
        noise = np.random.normal(0, 1,
                                size=(current_batch_size,) + (1, 1, 100))

        # Generate images
        generated_images = generator.predict(noise)

        # Add some noise to the labels that will be
        # fed to the discriminator
        real_output = (np.ones(current_batch_size) -
                      np.random.random_sample(current_batch_size) * 0.2)
        fake_output = np.random.random_sample(current_batch_size) * 0.2

        # Let's train the discriminator
        discriminator.trainable = True

        d_loss = discriminator.train_on_batch(real_images, real_output)
        d_loss += discriminator.train_on_batch(generated_images, fake_output)

        discriminator_loss = np.append(discriminator_loss, d_loss)

        # Now it's time to train the generator
        discriminator.trainable = False

        noise = np.random.normal(0, 1,
                                size=(current_batch_size * 2,) +
                                (1, 1, 100))

        # We try to mislead the discriminator by giving the opposite labels
        fake_output = (np.ones(current_batch_size * 2) -
                  np.random.random_sample(current_batch_size * 2) * 0.2)

        g_loss = gan.train_on_batch(noise, fake_output)
        adversarial_loss = np.append(adversarial_loss, g_loss)
        batches = np.append(batches, current_batch)

        # Each 5 batches show and save images
        if((batch_number + 1) % 5 == 0 and
            current_batch_size == batch_size):
            save_generated_images(generated_images, epoch, batch_number)

        time_elapsed = time.time() - start_time

        # Display and plot the results
        print("     Batch " + str(batch_number + 1) + "/" +
             str(number_of_batches) +
             " generator loss | discriminator loss : " +
             str(g_loss) + " | " + str(d_loss) + ' - batch took ' +
             str(time_elapsed) + ' s.')

        current_batch += 1

    # Save the model weights each 4 epochs
    if (epoch + 1) % 4 == 0:
        discriminator.trainable = True
        generator.save('output/generator_epoch' + str(epoch) + '.hdf5')
        discriminator.save('output/discriminator_epoch' +
                            str(epoch) + '.hdf5')
        display_images()
        
    # Each epoch update the loss graphs
    plt.figure(1)
    plt.plot(batches, adversarial_loss, color='green',
            label='Generator Loss')
    plt.plot(batches, discriminator_loss, color='blue',
            label='Discriminator Loss')
    plt.title("DCGAN Train")
    plt.xlabel("Batch Iteration")
    plt.ylabel("Loss")
    if epoch == 0:
        plt.legend()
    plt.pause(0.0000000001)
    plt.show()
    plt.savefig('trainingLossPlot.png')


In [ ]:
def main():
    dataset_path = "./dataset/class/"
    batch_size = 100
    image_shape = (64, 64, 3)
    epochs = 16000
    train_dcgan(batch_size, epochs,
                image_shape, dataset_path)

In [ ]:
if __name__ == "__main__":
    main()